In [3]:
import os
import json
import pandas as pd


In [4]:
with open("../utils/category_info.json") as f:
    category_info = json.load(f)

In [13]:
DATASET_NAME = "NEW_DATASET"
DATASET_PATH = f"E:/Giacomo/Tovanella/{DATASET_NAME}"

### Count samples divided in splits

In [ ]:
with open(f"../utils/{DATASET_NAME}/train_test_division.json") as f:
    train_test_div = json.load(f)

species_count = {}
for species_name, dates in train_test_div.items():
    species_count[species_name] = {"train": 0, "test": 0}
    species_count[species_name]["train"] = sum([date["count"] for date in dates.values() if date["is_training"] == True])
    species_count[species_name]["test"] = sum([date["count"] for date in dates.values() if date["is_training"] == False])

In [ ]:
species_count_df = pd.DataFrame(species_count).T
species_count_df.to_csv(f"../utils/{DATASET_NAME}/species_count_df.csv")

### Divides species audio path in "train" and "test"

In [8]:
species_split = {}
species_list = list(category_info.keys())
for species in species_list:
    species_split[species] = {"train": set(), "test": set()}

In [ ]:
split_count = "train"
for species in species_list:
    # check if removed
    split_path = os.path.join(DATASET_PATH, split_count, species)
    removed_path = os.path.join(DATASET_PATH, "removed", species)
    if os.path.exists(split_path):
        folder_path = split_path
    elif os.path.exists(removed_path):
        folder_path = removed_path
    else:
        print("Deleted species?", species)
        continue
    if species not in species_split:
        species_split[species][split] = set()
    for audio in os.listdir(folder_path):
        # 20190608_070000_64.wav
        date = audio.split("_")[0]
        is_training = train_test_div[species][date]["is_training"]
        split = "train" if is_training else "test"
        species_split[species][split].add(audio)

Deleted species? Prunella modularis_Dunnock
Deleted species? Spinus spinus_Eurasian Siskin


In [ ]:
split_count = "test"
for species in species_list:
    # check if removed
    split_path = os.path.join(DATASET_PATH, split_count, species)
    removed_path = os.path.join(DATASET_PATH, "removed", species)
    if os.path.exists(split_path):
        folder_path = split_path
    elif os.path.exists(removed_path):
        folder_path = removed_path
    else:
        print("Deleted species?", species)
        continue
    if species not in species_split:
        species_split[species][split] = set()
    for audio in os.listdir(folder_path):
        # 20190608_070000_64.wav
        date = audio.split("_")[0]
        is_training = train_test_div[species][date]["is_training"]
        split = "train" if is_training else "test"
        species_split[species][split].add(audio)


Deleted species? Prunella modularis_Dunnock
Deleted species? Spinus spinus_Eurasian Siskin


In [11]:
for species in species_split:
    species_split[species]["train"] = list(species_split[species]["train"])
    species_split[species]["test"] = list(species_split[species]["test"])

In [ ]:
# with open(f"../utils/{DATASET_NAME}/species_split.json", "w") as f:
#     json.dump(species_split, f)

### Get mean confidence score from all models for each audio in test

In [19]:
# get list of all models
CLASSIFIERS_PATH = "../classifiers/official/"
all_pred_test_segments = {}
for model in os.listdir(CLASSIFIERS_PATH):
    path_to_json = os.path.join(CLASSIFIERS_PATH, model, "test_complete_pred_segments.json")
    if not os.path.exists(path_to_json):
        continue
    with open(path_to_json) as f:
        pred_segments = json.load(f)
    all_pred_test_segments[model] = pred_segments

In [23]:
all_pred_test_segments

{'augm_bn': {'20190608_070000.WAV': {'62': {'Turdus philomelos_Song Thrush': 0.49516400694847107,
    'Erithacus rubecula_European Robin': 0.40817591547966003,
    'Fringilla coelebs_Common Chaffinch': 0.1490427851676941},
   '81': {'Erithacus rubecula_European Robin': 0.9590365290641785},
   '88': {'Turdus philomelos_Song Thrush': 0.2798563838005066,
    'Erithacus rubecula_European Robin': 0.19536025822162628},
   '-1': {'Fringilla coelebs_Common Chaffinch': 0.3945621848106384,
    'Turdus philomelos_Song Thrush': 0.289418488740921},
   '1': {'Turdus philomelos_Song Thrush': 0.44453856348991394,
    'Fringilla coelebs_Common Chaffinch': 0.22221174836158752,
    'Erithacus rubecula_European Robin': 0.17925502359867096},
   '103': {'Erithacus rubecula_European Robin': 0.7926658391952515,
    'Turdus philomelos_Song Thrush': 0.12627719342708588},
   '119': {'Erithacus rubecula_European Robin': 0.7010176181793213,
    'Turdus philomelos_Song Thrush': 0.35257458686828613},
   '12': {'Turd

In [29]:
from collections import defaultdict

aggregated_data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for model, audio_data in all_pred_test_segments.items():
    for audio, segments in audio_data.items():
        for segment, species_scores in segments.items():
            for species, score in species_scores.items():
                aggregated_data[audio][segment][species].append(score)


In [35]:
mean_conf_scores = {}
for audio, segments in aggregated_data.items():
    mean_conf_scores[audio] = {}
    for segment, species_scores in segments.items():
        mean_conf_scores[audio][segment] = {
            species: sum(scores) / len(scores) for species, scores in species_scores.items()
        }

mean_conf_scores["20190608_070000.WAV"]["62"]

{'Turdus philomelos_Song Thrush': 0.375798965493838,
 'Erithacus rubecula_European Robin': 0.41325686194679956,
 'Fringilla coelebs_Common Chaffinch': 0.16732944548130035,
 'Regulus ignicapilla_Common Firecrest': 0.11654996871948242,
 'Pseudonestor xanthophrys_Maui Parrotbill': 0.12227264791727066}

In [ ]:
# with open("../utils/mean_conf_scores.json", "w") as f:
#     json.dump(mean_conf_scores, f)